# Example use of carpenter (raw modules) with Dask
## Sequence definition
1. Load in data
2. Define a new variable
3. Fill a binned dataframe
5. apply a cut
6. fill another binned dataframe

In [1]:
%reload_ext autoreload
%autoreload 2


In [14]:
from fast_carpenter.data_import import Uproot4DataImport
from fast_carpenter.data_mapping import create_mapping

DATA_FILE = "../examples/tutorial/HEPTutorial/files/data.root"
OUTPUT_DIR = "../examples/tutorial/output"

data_import = Uproot4DataImport(None)
root_file = data_import.open([DATA_FILE])
data = create_mapping(
    root_file,
    treenames=["events"],
    methods="uproot4",
    connector="tree",
)
data.add_dataset_info("data", "data")


In [15]:
# content of root_file
for k in sorted(data.keys()):
    if k.startswith("N"):
        print(k)

print(data_import.config)
print(data.config)

NElectron
NJet
NMuon
NPhoton
NPrimaryVertices
{}
ConfigProxy(dataset=DatasetConfig(name='data', eventtype='data'))


In [16]:
from fast_carpenter.define import Define
define_jet_pt = Define(
    name="CreateJetPt",
    out_dir=OUTPUT_DIR,
    variables=[{"Jet_Pt": "sqrt(Jet_Px**2 + Jet_Py**2)"}],

)


In [22]:
from fast_carpenter.selection import CutFlow
cutflow = CutFlow(
    name="EventSelection",
    out_dir=OUTPUT_DIR,
    selection={
        "All": ["NJet >= 2"],
    },
    weights= {"weighted": "EventWeight"},
)


In [50]:
from fast_carpenter.summary import BinnedDataframe
jet_pt_hist = BinnedDataframe(
    name="JetPtHist",
    out_dir=OUTPUT_DIR,
    dataset_col=True,
    binning=[{'in': 'Jet_Pt', 'out': 'jet_pt_hist', 'bins': {'low': 20, 'high': 200, 'nbins': 90}}],
    weights= {"weighted": "EventWeight"},
)

In [51]:
sequence = [define_jet_pt, cutflow, jet_pt_hist]

In [53]:
define_jet_pt.event(data)

Trying to overwrite existing variable: Jet_Pt


True

In [54]:
cutflow.event(data)

In [55]:
jet_pt_hist.event(data)


True

In [31]:
collectors = [(s, s.collector()) for s in sequence if hasattr(s, "collector")]


In [32]:
collectors

[(<fast_carpenter.selection.stage.CutFlow at 0x7fd305147f70>,
  <fast_carpenter.selection.stage.Collector at 0x7fd304f95360>),
 (<fast_carpenter.summary.binned_dataframe.BinnedDataframe at 0x7fd304bad510>,
  <fast_carpenter.summary.binned_dataframe.Collector at 0x7fd304f96830>)]

In [34]:
a = collectors[0][0].selection

In [37]:
a.to_dataframe()

passed_only_cut             passed_incl              \
                               unweighted EventWeight  unweighted EventWeight   
unique_id depth cut                                                             
0         0     All                 26599     26599.0       26599     26599.0   
0,0       1     NJet >= 2           26599     26599.0       26599     26599.0   

                          totals_incl              
                           unweighted EventWeight  
unique_id depth cut                                
0         0     All            469384    469384.0  
0,0       1     NJet >= 2      469384    469384.0

In [41]:
b = collectors[1][0]


In [47]:
b._out_bin_dims


['jet_pt_hist']

In [43]:
b._binnings

[IntervalIndex([[-inf, 20.0), [20.0, 28.166666666666664), [28.166666666666664, 36.33333333333333), [36.33333333333333, 44.5), [44.5, 52.666666666666664) ... [967.3333333333333, 975.4999999999999), [975.4999999999999, 983.6666666666666), [983.6666666666666, 991.8333333333333), [991.8333333333333, 1000.0), [1000.0, inf)], dtype='interval[float64, left]', length=122)]

In [46]:
b._binnings[0]

IntervalIndex([[-inf, 20.0), [20.0, 28.166666666666664), [28.166666666666664, 36.33333333333333), [36.33333333333333, 44.5), [44.5, 52.666666666666664) ... [967.3333333333333, 975.4999999999999), [975.4999999999999, 983.6666666666666), [983.6666666666666, 991.8333333333333), [991.8333333333333, 1000.0), [1000.0, inf)], dtype='interval[float64, left]', length=122)

In [49]:
b.contents


,n
jet_pt_hist,
"[-inf, 20.0)",0
"[20.0, 28.166666666666664)",0
"[28.166666666666664, 36.33333333333333)",36966
"[36.33333333333333, 44.5)",35958
"[44.5, 52.666666666666664)",20805
...,...
"[967.3333333333333, 975.4999999999999)",0
"[975.4999999999999, 983.6666666666666)",0
"[983.6666666666666, 991.8333333333333)",0
